# **GenAI Aplicada ao Mercado Financeiro com LangChain**

**Autor**: Renan Santos Mendes

**Email**: renansantosmendes@gmail.com

**Descrição**: Este notebook demonstra aplicações práticas de Inteligência Artificial Generativa no mercado financeiro brasileiro usando LangChain e OpenAI. O projeto explora diferentes padrões de interação com LLMs para criar soluções financeiras inteligentes:

- **Análise de Ações**: Templates personalizados para análise técnica e fundamentalista de ativos
- **Chatbot Financeiro**: Sistema conversacional para consultoria de investimentos
- **Análise Completa de Portfólio**: Chains complexas para planejamento financeiro personalizado
- **Sistema de Alertas**: Geração automática de alertas sobre eventos de mercado

O notebook utiliza componentes fundamentais do LangChain como PromptTemplate, ChatPromptTemplate, Chains e diferentes tipos de mensagens (System, Human, AI) para construir aplicações robustas e escaláveis no contexto financeiro.

## **Instalação dos Pacotes**

In [3]:
%%capture
!pip install langchain langchain-openai

## **Configuração das Variáveis de Ambiente**

Configuração da chave de API da OpenAI para autenticação

In [4]:
import os

from google.colab import userdata


os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')

## **Imports dos Pacotes Usados no Notebook**

Importação dos componentes principais do LangChain para construção de aplicações com LLMs

In [5]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

## **Inicialização do Modelo LLM**

Criação da instância do modelo ChatGPT que será utilizado em todos os exemplos

In [6]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=os.getenv("OPENAI_API_KEY")
)

print("Modelo LLM inicializado com sucesso!")

Modelo LLM inicializado com sucesso!


## **Exemplo 1: Análise de Ações com PromptTemplate**

Este exemplo demonstra como criar um template parametrizável para análise de ações. O PromptTemplate permite definir variáveis dinâmicas (ticker, preço, período) que são substituídas em tempo de execução, facilitando a reutilização do mesmo prompt para diferentes ativos.

In [7]:
stock_analysis_template = """
Como analista financeiro experiente, forneça uma análise detalhada da ação {ticker}.

Considere os seguintes aspectos:
- Setor da empresa
- Principais métricas financeiras relevantes
- Riscos e oportunidades
- Recomendação de investimento (compra, venda, manter)

Preço atual: R$ {price}
Período de análise: {period}

Análise:
"""

stock_prompt = PromptTemplate(
    input_variables=["ticker", "price", "period"],
    template=stock_analysis_template
)

formatted_prompt = stock_prompt.format(
    ticker="PETR4",
    price="38.50",
    period="último trimestre"
)

response = llm.invoke(formatted_prompt)

print("=== ANÁLISE DE AÇÃO PETR4 ===")
print(response.content)

=== ANÁLISE DE AÇÃO PETR4 ===
### Análise da Ação PETR4

**Setor da Empresa:**
A PETR4 refere-se às ações preferenciais da Petrobras, uma das maiores empresas de energia do Brasil e uma das maiores do mundo em petróleo e gás. A empresa está inserida no setor de energia, mais especificamente na exploração, produção e refino de petróleo e gás natural. O setor é caracterizado por ser altamente regulado e volátil, influenciado por fatores como os preços internacionais do petróleo, políticas governamentais, e questões ambientais.

**Principais Métricas Financeiras Relevantes:**
1. **Preço/Lucro (P/L):** Um indicador importante para avaliar se a ação está supervalorizada ou subvalorizada. O P/L da PETR4 pode ser comparado ao de outras empresas do setor e à média do mercado.
   
2. **Dividend Yield:** A Petrobras historicamente tem um bom histórico de pagamento de dividendos, o que pode ser atraente para investidores em busca de renda passiva.
   
3. **Endividamento:** A relação dívida/EBITDA

## **Exemplo 2: Chatbot Financeiro com ChatPromptTemplate**

Este exemplo implementa um chatbot financeiro usando ChatPromptTemplate, que permite estruturar conversas com diferentes tipos de mensagens (System, Human, AI). A mensagem System define o comportamento e expertise do assistente, enquanto as mensagens Human representam as perguntas dos usuários.

In [8]:
chat_template = ChatPromptTemplate.from_messages([
    SystemMessage(
        content="""Você é um consultor financeiro especializado no mercado brasileiro.
        Forneça conselhos práticos e baseados em princípios sólidos de investimento.
        Sempre mencione os riscos envolvidos e não dê garantias de retorno."""
    ),
    HumanMessage(content="{user_question}")
])

questions = [
    "Como devo começar a investir com R$ 1.000?",
    "Qual a diferença entre CDB e Tesouro Direto?",
    "Vale a pena investir em criptomoedas?"
]

print("=== CHATBOT FINANCEIRO ===")
for i, question in enumerate(questions, 1):
    messages = chat_template.format_messages(user_question=question)
    response = llm.invoke(messages)

    print(f"\n{i}. Pergunta: {question}")
    print(f"Resposta: {response.content}")
    print("-" * 80)

=== CHATBOT FINANCEIRO ===

1. Pergunta: Como devo começar a investir com R$ 1.000?
Resposta: Parece que você não colocou uma pergunta específica. Por favor, forneça mais detalhes ou uma questão específica sobre finanças ou investimentos que você gostaria de discutir, e ficarei feliz em ajudar!
--------------------------------------------------------------------------------

2. Pergunta: Qual a diferença entre CDB e Tesouro Direto?
Resposta: Parece que você não incluiu uma pergunta específica. Por favor, forneça mais detalhes ou uma pergunta sobre investimentos ou finanças que você gostaria de discutir, e ficarei feliz em ajudar!
--------------------------------------------------------------------------------

3. Pergunta: Vale a pena investir em criptomoedas?
Resposta: Parece que você não fez uma pergunta específica. Sinta-se à vontade para perguntar sobre investimentos, planejamento financeiro, estratégias de economia ou qualquer outro assunto relacionado às finanças. Estou aqui para

## **Exemplo 3: Chain para Análise Completa de Investimento**

Este exemplo demonstra o uso de Chains do LangChain, que permitem encadear prompts e modelos de forma declarativa usando o operador `|`. A chain combina o template de análise completa com o LLM, criando um pipeline reutilizável que pode ser invocado com diferentes parâmetros de investidor.

In [9]:
complete_analysis_template = """
Realize uma análise completa de investimento para o seguinte cenário:

Perfil do Investidor: {profile}
Capital Disponível: R$ {capital}
Objetivo: {goal}
Prazo: {timeframe}
Tolerância ao Risco: {risk_tolerance}

Forneça:
1. Estratégia de investimento recomendada
2. Alocação de ativos sugerida
3. Produtos financeiros específicos
4. Cronograma de aportes
5. Principais riscos e como mitigá-los

Análise:
"""

complete_prompt = PromptTemplate(
    input_variables=["profile", "capital", "goal", "timeframe", "risk_tolerance"],
    template=complete_analysis_template
)

analysis_chain = complete_prompt | llm

result = analysis_chain.invoke({
    "profile": "Jovem profissional, 28 anos",
    "capital": "50.000",
    "goal": "Aposentadoria",
    "timeframe": "35 anos",
    "risk_tolerance": "Moderada"
})

print("=== ANÁLISE COMPLETA DE INVESTIMENTO ===")
print(result.content)

=== ANÁLISE COMPLETA DE INVESTIMENTO ===
### Análise Completa de Investimento

**Perfil do Investidor:**
- **Idade:** 28 anos
- **Capital Disponível:** R$ 50.000
- **Objetivo:** Aposentadoria
- **Prazo:** 35 anos
- **Tolerância ao Risco:** Moderada

### 1. Estratégia de Investimento Recomendada

Dado o perfil do investidor, a estratégia de investimento deve focar em um crescimento sustentável do capital ao longo do tempo, aproveitando o horizonte de longo prazo para mitigar flutuações de mercado. O investidor deve diversificar seus investimentos para equilibrar risco e retorno, alocando uma parte significativa em ativos de renda variável, mas também garantindo uma base sólida em renda fixa.

### 2. Alocação de Ativos Sugerida

- **Renda Variável (50%):** R$ 25.000
  - Ações de empresas de grande capitalização (30%): R$ 15.000
  - Fundos de índice (ETFs) focados em ações (20%): R$ 10.000

- **Renda Fixa (30%):** R$ 15.000
  - Títulos do Tesouro Direto (Tesouro Selic e Tesouro IPCA): R$ 

## **Exemplo 4: Sistema de Alertas de Mercado**

Este exemplo implementa um sistema automatizado de geração de alertas sobre eventos de mercado. A função encapsula a lógica de criação de prompts dinâmicos e pode ser facilmente integrada em sistemas de monitoramento em tempo real, gerando análises contextualizadas para diferentes tipos de eventos econômicos.

In [10]:
def generate_market_alert(event, impact):
    prompt = f"""
    Como analista de mercado, gere um alerta para investidores sobre o seguinte evento:

    Evento: {event}
    Impacto Estimado: {impact}

    O alerta deve conter:
    - Resumo do evento
    - Impacto esperado nos mercados
    - Setores/ativos mais afetados
    - Ações recomendadas para investidores
    - Nível de urgência (baixo/médio/alto)

    Seja conciso e objetivo.
    """

    response = llm.invoke(prompt)
    return response.content


events = [
    ("Decisão do COPOM sobre taxa Selic", "Moderado - possível alta de 0,5%"),
    ("Divulgação do PIB trimestral", "Baixo - resultado dentro do esperado"),
    ("Crise geopolítica internacional", "Alto - possível fuga de capital")
]

print("=== SISTEMA DE ALERTAS DE MERCADO ===")
for i, (event, impact) in enumerate(events, 1):
    alert = generate_market_alert(event, impact)

    print(f"\n--- ALERTA {i} ---")
    print(alert)
    print("-" * 80)

=== SISTEMA DE ALERTAS DE MERCADO ===

--- ALERTA 1 ---
**Alerta para Investidores: Decisão do COPOM sobre Taxa Selic**

**Resumo do Evento:**  
O Comitê de Política Monetária (COPOM) do Banco Central do Brasil se reunirá em breve para decidir sobre a taxa Selic. A expectativa é de uma alta de 0,5%.

**Impacto Esperado nos Mercados:**  
Uma alta na Selic tende a encarecer o crédito, impactando o consumo e os investimentos. O mercado pode apresentar volatilidade, com potencial pressão sobre ações e títulos de renda fixa.

**Setores/Ativos Mais Afetados:**  
- **Setor Financeiro:** Bancos podem se beneficiar com margens de juros mais altas.  
- **Setor de Consumo:** Empresas dependentes de crédito podem ser negativamente impactadas.  
- **Setor Imobiliário:** Pode enfrentar dificuldades devido ao aumento nos custos de financiamento.

**Ações Recomendadas para Investidores:**  
1. Avaliar a exposição a ações de empresas do setor de consumo e imobiliário.  
2. Considerar a diversificação e

## **Exemplo 5: Análise de Portfólio com Structured Output**

Este exemplo demonstra o uso de **Structured Output** do LangChain para garantir que as respostas do LLM sigam um formato estruturado e validado. Utilizamos Pydantic models para definir o schema da resposta, garantindo que a análise de portfólio retorne dados consistentes e tipados que podem ser facilmente processados programaticamente.

A estrutura de dados define campos obrigatórios como percentuais de alocação, score de risco e recomendações específicas, eliminando a necessidade de parsing manual das respostas do modelo.


In [12]:
from typing import List
from pydantic import BaseModel, Field


class AssetAllocation(BaseModel):
    asset_class: str = Field(description="Classe do ativo (Renda Fixa, Ações, FIIs, etc)")
    percentage: float = Field(description="Percentual de alocação recomendado")
    rationale: str = Field(description="Justificativa para esta alocação")


class PortfolioAnalysis(BaseModel):
    investor_profile: str = Field(description="Perfil do investidor identificado")
    risk_score: int = Field(description="Score de risco de 1 a 10")
    allocations: List[AssetAllocation] = Field(description="Lista de alocações recomendadas")
    total_expected_return: float = Field(description="Retorno anual esperado em percentual")
    key_recommendations: List[str] = Field(description="Principais recomendações")


structured_llm = llm.with_structured_output(PortfolioAnalysis)

portfolio_prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um planejador financeiro certificado.
    Analise o perfil do investidor e forneça uma recomendação de portfólio estruturada."""),
    ("human", """Investidor:
    - Idade: {age} anos
    - Capital: R$ {capital}
    - Objetivo: {goal}
    - Prazo: {timeframe}
    - Experiência: {experience}

    Forneça uma análise completa de portfólio.""")
])

portfolio_chain = portfolio_prompt | structured_llm

analysis = portfolio_chain.invoke({
    "age": "35",
    "capital": "100.000",
    "goal": "Independência financeira",
    "timeframe": "15 anos",
    "experience": "Intermediário"
})

print("=== ANÁLISE DE PORTFÓLIO ESTRUTURADA ===\\n")
print(f"Perfil: {analysis.investor_profile}")
print(f"Score de Risco: {analysis.risk_score}/10")
print(f"Retorno Esperado: {analysis.total_expected_return}% ao ano\\n")

print("Alocação Recomendada:")
for allocation in analysis.allocations:
    print(f"  • {allocation.asset_class}: {allocation.percentage}%")
    print(f"    Justificativa: {allocation.rationale}\\n")

print("Recomendações Principais:")
for i, rec in enumerate(analysis.key_recommendations, 1):
    print(f"  {i}. {rec}")

=== ANÁLISE DE PORTFÓLIO ESTRUTURADA ===\n
Perfil: Moderado
Score de Risco: 6/10
Retorno Esperado: 8.0% ao ano\n
Alocação Recomendada:
  • Ações: 40.0%
    Justificativa: Ações têm potencial de crescimento a longo prazo e podem oferecer retornos significativos, alinhando-se ao objetivo de independência financeira.\n
  • Renda Fixa: 30.0%
    Justificativa: A alocação em renda fixa proporciona uma maior segurança e estabilidade ao portfólio, ajudando a balancear a volatilidade das ações.\n
  • Fundos Imobiliários (FIIs): 15.0%
    Justificativa: Os FIIs oferecem uma boa oportunidade de renda passiva e valorização, diversificando ainda mais o portfólio.\n
  • Fundos de Investimento em Ações: 10.0%
    Justificativa: Os fundos de investimento em ações permitem diversificação em ações sem necessidade de gerenciamento ativo, ideal para um investidor intermediário.\n
  • Criptoativos: 5.0%
    Justificativa: Uma pequena alocação em criptoativos pode proporcionar alto potencial de retorno, em

## **Exemplo 6: Sistema de Análise Multi-Step com RunnablePassthrough**

Este exemplo implementa um pipeline complexo usando **RunnablePassthrough** e **RunnableLambda** para criar um sistema de análise financeira em múltiplas etapas. O fluxo passa por três estágios sequenciais:

1. **Análise Fundamental**: Avalia métricas financeiras e fundamentos da empresa
2. **Análise Técnica**: Examina padrões de preço e indicadores técnicos
3. **Síntese Final**: Combina ambas análises em uma recomendação consolidada

O RunnablePassthrough permite que dados sejam passados entre etapas mantendo o contexto completo, enquanto RunnableLambda possibilita transformações customizadas dos dados intermediários.



In [14]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


fundamental_prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um analista fundamentalista.
    Analise apenas os fundamentos financeiros da empresa."""),
    ("human", """Empresa: {company}
    Setor: {sector}
    P/L: {pe_ratio}
    ROE: {roe}%
    Dividend Yield: {dividend_yield}%

    Forneça análise fundamentalista concisa.""")
])

technical_prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um analista técnico.
    Analise apenas os aspectos técnicos do ativo."""),
    ("human", """Empresa: {company}
    Preço Atual: R$ {current_price}
    Máxima 52 semanas: R$ {high_52w}
    Mínima 52 semanas: R$ {low_52w}
    Volume Médio: {avg_volume}

    Análise Fundamentalista Prévia: {fundamental_analysis}

    Forneça análise técnica concisa.""")
])

synthesis_prompt = ChatPromptTemplate.from_messages([
    ("system", """Você é um estrategista de investimentos.
    Sintetize as análises fundamental e técnica em uma recomendação final."""),
    ("human", """Empresa: {company}

    Análise Fundamentalista:
    {fundamental_analysis}

    Análise Técnica:
    {technical_analysis}

    Forneça recomendação final: COMPRA FORTE, COMPRA, MANTER, VENDA ou VENDA FORTE.
    Justifique com base em ambas análises.""")
])


def extract_content(ai_message):
    return ai_message.content


multi_step_chain = (
    RunnablePassthrough.assign(
        fundamental_analysis=fundamental_prompt | llm | RunnableLambda(extract_content)
    )
    | RunnablePassthrough.assign(
        technical_analysis=technical_prompt | llm | RunnableLambda(extract_content)
    )
    | RunnablePassthrough.assign(
        final_recommendation=synthesis_prompt | llm | RunnableLambda(extract_content)
    )
)

stock_data = {
    "company": "Vale S.A. (VALE3)",
    "sector": "Mineração",
    "pe_ratio": "4.5",
    "roe": "18.5",
    "dividend_yield": "9.2",
    "current_price": "62.50",
    "high_52w": "72.80",
    "low_52w": "55.20",
    "avg_volume": "45.000.000"
}

result = multi_step_chain.invoke(stock_data)

print("=== ANÁLISE MULTI-STEP VALE3 ===\\n")
print("📊 ANÁLISE FUNDAMENTALISTA:")
print(result["fundamental_analysis"])
print("\\n" + "="*80 + "\\n")

print("📈 ANÁLISE TÉCNICA:")
print(result["technical_analysis"])
print("\\n" + "="*80 + "\\n")

print("🎯 RECOMENDAÇÃO FINAL:")
print(result["final_recommendation"])

=== ANÁLISE MULTI-STEP VALE3 ===\n
📊 ANÁLISE FUNDAMENTALISTA:
A análise fundamentalista da Vale S.A. (VALE3) apresenta alguns pontos positivos e negativos que merecem consideração.

**P/L (Preço sobre Lucro):** Com um P/L de 4.5, a Vale está sendo negociada a um múltiplo relativamente baixo em comparação com o mercado em geral. Isso pode indicar que as ações estão subvalorizadas, sugerindo uma oportunidade de compra, ou pode refletir preocupações com o futuro da empresa.

**ROE (Retorno sobre Patrimônio Líquido):** Um ROE de 18.5% é um indicador forte de que a empresa está gerando um bom retorno sobre o capital dos acionistas. Isso sugere eficiência na utilização dos recursos e pode atrair investidores que buscam empresas com forte capacidade de geração de lucros.

**Dividend Yield:** O Dividend Yield de 9.2% é bastante atrativo, especialmente em um cenário de juros baixos, tornando a Vale uma opção interessante para investidores que buscam renda passiva. Esse nível de dividendos suger